# Part 2: Hardware Demo
## Outline
1. RPC server and client  
2. Signal generation  
    2.1 Pulse generation  
    2.2 Gate generation  
    2.3 Circuit generation  
3. Loopback test  
4. Readout emulator and GMM fitting  
5. Fast feedback - Active reset  

## 1. RPC server and client

![infra.svg](./images/infra.svg)

In [ ]:
import qubic.toolchain as tc
import qubic.rpc_client as rc
import qubitconfig.qchip as qc
from distproc.hwconfig import FPGAConfig, load_channel_configs
import chipcalibration.config as cfg
import numpy as np
import matplotlib.pyplot as plt
import qubic.state_disc as sd
from chipcalibration import vna as vn
import qubic.job_manager as jm

## 2. Signal generation
### 2.1 Pulse generation

![cirgen.svg](./images/cirgen.svg)

### Load configs and define circuit.
Using the chipcalibration repository, load all three configs:  
a. FPGA config: provides timing information for the scheduler  
b. Channel configs: firmware channel mapping + configuration, see [Understanding Channel Configuration](https://gitlab.com/LBL-QubiC/software/-/wikis/Understanding-Channel-Configuration) for details  
c. QChip object: contains calibrated gates + readout  

In [ ]:
fpga_config = FPGAConfig()
channel_configs = load_channel_configs('channel_config.json')
qchip = qc.QChip('qubitcfg.json')
qchip.cfg_dict

Define a circuit at the pulse-level. For details on the QubiC circuit format and supported operations, see [compiler.py](https://gitlab.com/LBL-QubiC/distributed_processor/-/blob/master/python/distproc/compiler.py).

In [ ]:
circuit_1 = [
    
    # play a pi pulse on Q3
    {'name': 'pulse', 'phase': 0, 'freq': 400e6, 'amp': 0.99, 'twidth': 64e-9,
     'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.25}},
     'dest': 'Q3.qdrv'},
    
    # play a pi/2 pulse on Q6
    {'name': 'pulse', 'phase': 0, 'freq': 5.7e9, 'amp': 0.50, 'twidth': 32e-9,
    'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.25}},
    'dest': 'Q6.qdrv'}

]

### Compile and assemble.

Compile the program. The output of the compile stage is a distributed processor assembly program, which consists of initialization / termination statements, as well as a list of scheduled pulses for each core.

In [ ]:
compiled_prog = tc.run_compile_stage(circuit_1, fpga_config, qchip)
compiled_prog.program

Run the assembler to convert the above program into machine code that we can load onto the FPGA.

In [ ]:
raw_asm = tc.run_assemble_stage(compiled_prog, channel_configs)

### Connect to server and run circuit.

Now that we have defined our circuit and compiled it to machine code, we can submit it to the ZCU216 and run it.

Instantiate the runner client.

In [ ]:
runner = rc.CircuitRunnerClient(ip='', port=9096)

Submit the circuit to the server, and collect 1 shot. The runner will run the currently loaded program (or a batch of circuits) and acquire the results from acq buf or acc buf.

In [ ]:
acq_data = runner.load_and_run_acq(raw_asm, n_total_shots=1, acq_chans=['0','1'], trig_delay=0e-9)

Observe the pulses through the acq buffer or on the oscilloscope.

In [ ]:
%matplotlib notebook
plt.xlabel('Time (s)')
plt.ylabel('ADC Counts')
plt.plot(np.arange(0,acq_data['1'].shape[1]*0.5e-9,0.5e-9)[10:], np.average(acq_data['1'],axis=0)[10:])

### 2.2 Gate generation
Define a circuit with calibrated gates / parameters.

In [ ]:
circuit_2 = [
    
    # play a pi/2 pulse on Q3
    {'name': 'X90', 'qubit': 'Q3', 'modi':{(0, 'amp'): 0.99, (0, 'freq'): 400e6}},
    
    # play two pi/2 pulses on Q3 and Q6 simultaneously
    {'name': 'X90', 'qubit': 'Q3'},
    {'name': 'X90', 'qubit': 'Q6'},
    
    # add delay
    {'name': 'delay', 't': 200.e-9}, 
    
    # play a CNOT gate on Q3 and Q6
    {'name': 'CNOT', 'qubit': ['Q3','Q6']},
    
    # schedule barrier
    {'name': 'barrier', 'qubit': ['Q3','Q6']},
    
    # play readout gates for measurement
    {'name': 'read', 'qubit': 'Q3'},
    {'name': 'read', 'qubit': 'Q6'}
    
]

In [ ]:
compiled_prog = tc.run_compile_stage(circuit_2, fpga_config, qchip)
raw_asm = tc.run_assemble_stage(compiled_prog, channel_configs)

In [ ]:
acq_data = runner.load_and_run_acq(raw_asm, n_total_shots=1, acq_chans=['0','1'], trig_delay=0e-9)
plt.figure()
plt.xlabel('Time (s)')
plt.ylabel('ADC Counts')
plt.plot(np.arange(0,acq_data['1'].shape[1]*0.5e-9,0.5e-9)[10:], np.average(acq_data['1'],axis=0)[10:])

### 2.3 Circuit generation
Now it is the time to create your own quantum circuit and capture it through the acq buffer.

In [ ]:
circuit_3 = []

In [ ]:
compiled_prog = tc.run_compile_stage(circuit_3, fpga_config, qchip)
raw_asm = tc.run_assemble_stage(compiled_prog, channel_configs)

In [ ]:
acq_data = runner.load_and_run_acq(raw_asm, n_total_shots=1, acq_chans=['0','1'], trig_delay=0e-9)
plt.figure()
plt.xlabel('Time (s)')
plt.ylabel('ADC Counts')
plt.plot(np.arange(0,acq_data['1'].shape[1]*0.5e-9,0.5e-9)[10:], np.average(acq_data['1'],axis=0)[10:])

## 3. Loopback test
Perform frequency sweeps while obtaining amplitude and phase responses in loopback mode, resembling the capabilities of a vector network analyzer (VNA). This dedicated tool is instrumental for conducting qubit readout spectroscopy. 

![vna.svg](./images/vna.svg)

In [ ]:
amp = 0.99
freqs = np.linspace(1.0e9, 4.0e9, 100)
nshots = 10
vna = vn.Vna(amp, freqs, nshots)
jobman = jm.JobManager(fpga_config, channel_configs, runner, qchip)
vna.run_and_report(jobman)

JobManager class is for compiling and executing circuits, which contains necessary config objects for compilation, runner for execution, and (optionally) GMMManager for state classification.

In [ ]:
fig, ax1 = plt.subplots()
ax1.set_xlabel('Frequency (Hz)')
ax1.set_ylabel('Amplitude (a.u.)', color='b')
ax1.plot(vna.freqs, vna.results['amp'], color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax2 = ax1.twinx()
ax2.set_ylabel('Phase (rad)', color='r')
ax2.plot(freqs, vna.results['phase'], color='r')
ax2.tick_params(axis='y', labelcolor='r')

## 4. Readout emulator and GMM fitting
### Create a qubit readout emulator with RF components to mimic the quantum signal processing.

![readout_emulator.svg](./images/readout_emulator.svg)

In [ ]:
fread=2.7568e9
circuit_4 = [
    {'name': 'read', 'qubit': 'Q3', 
     'modi':{(0, 'amp'): 0.99, (0, 'freq'): fread, (1, 'freq'): fread, (1, 'phase'): 0}}
]

In [ ]:
compiled_prog = tc.run_compile_stage(circuit_4, fpga_config, qchip)
raw_asm = tc.run_assemble_stage(compiled_prog, channel_configs)
s11 = runner.run_circuit_batch([raw_asm], 2000, delay_per_shot=0)

A dictionary of downconverted + integrated complex (IQ) values is returned for each loaded channel. Here, we are using Q3, so we get back data for channel '3'.

In [ ]:
plt.figure()
ax1 = plt.subplot(111)
ax1.set_aspect('equal')
plt.plot(s11['3'].real[0], s11['3'].imag[0], '.')
lim = max(1.1*max(max(abs(s11['3'].real[0])), max(abs(s11['3'].imag[0]))), 0.1)
ax1.set_xlim([-lim,lim])
ax1.set_ylim([-lim,lim])
ax1.set_xlabel('I (a.u.)')
ax1.set_ylabel('Q (a.u.)')
plt.grid()

### Fit the two blobs with Gaussian Mixture Model (GMM).

In [ ]:
gmm_manager = sd.GMMManager(chanmap_or_chan_cfgs=channel_configs)
gmm_manager.fit(s11)
gmm_manager.gmm_dict['Q3'].gmmfit.means_

### Rotate the blobs on the IQ plane to create a decision boundary along the Y-axis.

In [ ]:
angle = gmm_manager.get_threshold_angle('Q3')
circuit_5 = [
    {'name': 'read', 'qubit': 'Q3',
     'modi':{(0, 'amp'): 0.99, (0, 'freq'): fread, (1, 'freq'): fread, (1, 'phase'): np.pi/2-angle}}
]

In [ ]:
compiled_prog = tc.run_compile_stage(circuit_5, fpga_config, qchip)
raw_asm = tc.run_assemble_stage(compiled_prog, channel_configs)
s11 = runner.run_circuit_batch([raw_asm], 2000, delay_per_shot=0)

In [ ]:
gmm_manager = sd.GMMManager(chanmap_or_chan_cfgs=channel_configs)
gmm_manager.fit(s11)
plt.figure()
ax1=plt.subplot(111)
ax1.set_aspect('equal')
plt.plot(s11['3'].real[0], s11['3'].imag[0], '.')
lim=max(1.1*max(max(abs(s11['3'].real[0])),max(abs(s11['3'].imag[0]))),0.1)
ax1.set_xlim([-lim,lim])
ax1.set_ylim([-lim,lim])
ax1.set_xlabel('I (a.u.)')
ax1.set_ylabel('Q (a.u.)')
plt.axvline(x=0, color='r')
plt.grid()

## 5. Fast feedback - Active reset
Active reset is a fast feedback technique utilized to swiftly and efficiently restore a quantum system to a well-defined initial state, commonly the ground state. The method involves a single-shot measurement of the qubit state, followed by a conditional single-qubit gate operation. If the qubit is found in an excited state, this gate operation rotates it into the ground state.
### Implement active reset circuit.

In [ ]:
cond_lhs = 1 if gmm_manager.gmm_dict['Q3'].gmmfit.means_[0][0]>0 else 0
circuit_6 = [
    {'name': 'X90', 'qubit': 'Q3', 'modi':{(0, 'amp'): 0.19, (0, 'freq'): 900e6}},
    {'name': 'read', 'qubit': 'Q3', 
     'modi':{(0, 'amp'): 0.99, (0, 'freq'): fread, (1, 'freq'): fread, (1, 'phase'): np.pi/2-angle}},
    {'name': 'branch_fproc', 'alu_cond': 'eq', 'cond_lhs': cond_lhs, 'func_id': 'Q3.meas', 'scope': 'Q3',
                'true': [{'name': 'delay', 't': 200.e-9, 'qubit': 'Q3'},
                         {'name': 'X90', 'qubit': 'Q3', 'modi':{(0, 'amp'): 0.99, (0, 'freq'): 900e6}}, 
                         {'name': 'X90', 'qubit': 'Q3', 'modi':{(0, 'amp'): 0.99, (0, 'freq'): 900e6}}], 
                'false': []},
]

### Perform single-shot measurement.

In [ ]:
compiled_prog = tc.run_compile_stage(circuit_6, fpga_config, qchip)
raw_asm = tc.run_assemble_stage(compiled_prog, channel_configs)
acq_data = runner.load_and_run_acq(raw_asm, n_total_shots=1, acq_chans=['0','1'], trig_delay=0e-9, return_acc=True)
print(acq_data[1])
gmm_manager.predict(acq_data[1])

### Check the conditional gate operation through the acq buffer.

In [ ]:
plt.figure()
plt.xlabel('Time (s)')
plt.ylabel('ADC Counts')
plt.plot(np.arange(0,acq_data[0]['1'].shape[1]*0.5e-9,0.5e-9)[10:], np.average(acq_data[0]['1'],axis=0)[10:])